# ING Lion's Den 2024


## Team: RiskBusters

 Michał Bryzik, Michał Niegierewicz, Kacper Gruca, Jan Ślusarek

In [373]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [374]:
data=pd.read_csv('../data/input/development_sample.csv') 
data.head() 

,ID,customer_id,application_date,target,Application_status,Var1,Var2,Var3,Var4,Var5,...,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,_r_
0,11034977,32537148,01Feb2010 0:00:00,0.0,Approved,1,2.0,1,7800,99,...,1,1,0,6768.42,0.00,0,0,0,3899,0.267045
1,11034978,32761663,01Feb2010 0:00:00,0.0,Approved,1,1.0,2,11100,78,...,0,0,15000,5937.66,3870.60,0,0,0,3899,0.533040
2,11034979,32701063,01Feb2010 0:00:00,0.0,Approved,2,3.0,1,2400,15,...,2,2,0,5647.77,1463.08,0,0,10,3899,0.861555
3,11034980,32386786,01Feb2010 0:00:00,0.0,Approved,3,1.0,2,11800,30,...,0,0,0,3594.90,21563.78,0,0,10,3899,0.506451
4,11034981,32692110,02Feb2010 0:00:00,NaN,Rejected,1,2.0,1,10200,72,...,2,2,0,9908.45,0.00,0,0,20,3899,0.210913


In [375]:
data.shape 

(50000, 36)

In [376]:
data.isnull().sum() 

ID                        0
customer_id               0
application_date          0
target                13282
Application_status        0
Var1                      0
Var2                   1403
Var3                   1403
Var4                      0
Var5                      0
Var6                      0
Var7                      0
Var8                  29173
Var9                      0
Var10                 37538
Var11                     0
Var12                 37538
Var13                     0
Var14                     0
Var15                     0
Var16                     0
Var17                    40
Var18                 37416
Var19                 29173
Var20                     0
Var21                     0
Var22                     0
Var23                     0
Var24                     0
Var25                 10088
Var26                 19883
Var27                     0
Var28                     0
Var29                     0
Var30                     0
_r_                 

In [377]:
data.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  50000 non-null  int64  
 1   customer_id         50000 non-null  int64  
 2   application_date    50000 non-null  object 
 3   target              36718 non-null  float64
 4   Application_status  50000 non-null  object 
 5   Var1                50000 non-null  int64  
 6   Var2                48597 non-null  float64
 7   Var3                48597 non-null  object 
 8   Var4                50000 non-null  int64  
 9   Var5                50000 non-null  int64  
 10  Var6                50000 non-null  int64  
 11  Var7                50000 non-null  float64
 12  Var8                20827 non-null  float64
 13  Var9                50000 non-null  int64  
 14  Var10               12462 non-null  float64
 15  Var11               50000 non-null  int64  
 16  Var1

### Sort columns by datetypes

In [378]:
# Define columns
date_columns = ['application_date', 'Var13'] 
numeric_columns = ['ID', 'customer_id', '_r_', 'Var9', 'Var10']  
categorical_columns = []
binary_columns = ['target', 'Application_status'] 

In [379]:
# Modify date columns
data_h=data.copy()

data_h['application_date'] = pd.to_datetime(data_h['application_date'], errors='coerce')# Conversion of date for application_date
data_h['Var13'] = pd.to_datetime(data_h['Var13'], format='%d%b%Y', errors='coerce') # Var13 conversion for irregular type 

In [380]:
# Checking the data type for the rest of columns 
for column in data_h.columns:
    if column in date_columns + numeric_columns + binary_columns:
        continue  
    elif column in ['Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']:
        numeric_columns.append(column)
    elif column in ['Var2', 'Var3', 'Var11', 'Var12', 'Var14']:
        categorical_columns.append(column)
    elif column in ['Var18', 'Var19', 'Var27', 'Var28']:
        binary_columns.append(column)

# Assumed sequence for columns sorting 
sorted_columns = date_columns + numeric_columns + categorical_columns + binary_columns

# Choose of the sorted value by defined categories
sorted_data = data_h[sorted_columns]

In [381]:
sorted_data.head()
# Some of the records for Var14 are Nulls, it requires analysis

,application_date,Var13,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,2010-02-01,2004-11-26,11034977,32537148,0.267045,5640,NaN,1,7800,99,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,2010-02-01,1996-09-16,11034978,32761663,0.533040,3000,NaN,1,11100,78,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,2010-02-01,1989-05-06,11034979,32701063,0.861555,6240,4300.0,2,2400,15,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,2010-02-01,1987-12-06,11034980,32386786,0.506451,4560,2500.0,3,11800,30,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,2010-02-02,1989-05-18,11034981,32692110,0.210913,5160,NaN,1,10200,72,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


### Alteration for date variables and creating associated column

In [382]:
data_h['Var13'].isnull().sum(0)

784

In [383]:
sorted_data['application_date'].isnull().sum(0)

0

In [384]:
 # We create a copy of the DataFrame 'data' to preserve the original data. This is a common practice
# when manipulating data to ensure that the original dataset remains unchanged for verification or other analysis purposes.
data_copy = data.copy()

# Convert 'Var13' to a datetime type in the copy, storing the result in a new column. The format '%d%b%Y' specifies
# the expected date format, such as '01Jan2020'. 'errors='coerce'' tells pandas to mark any values that don't match
# this format as NaT (Not a Time), which is used for missing or erroneous date values.
data_copy['Var13_converted'] = pd.to_datetime(data_copy['Var13'], format='%d%b%Y', errors='coerce')

# Identify indices where the conversion resulted in 'NaT', indicating missing values. This step is crucial for
# identifying data points that may have been entered incorrectly or are missing.
nan_indices = data_copy['Var13_converted'].isna()

# Compare the original 'Var13' values that turned into 'NaT' after conversion. This comparison is useful for
# diagnosing issues with the data conversion, allowing for a targeted review of problematic data entries.
original_values_with_nan = data_copy.loc[nan_indices, 'Var13']

print("Original 'Var13' values that became empty after conversion:")
print(original_values_with_nan)

# Attempt to convert these dates in a more flexible manner to account for possible anomalies. This step is aimed at
# capturing dates that may not strictly adhere to the specified format but can still be considered valid dates with a
# more lenient parsing strategy.
# We filter out those that are not 'NaT' and check if they are beyond the year 2024. This can help in identifying
# outlier or future dates that may not be relevant or correct for the analysis.
dates_after_2024 = original_values_with_nan.apply(
    lambda x: pd.to_datetime(x, format='%d%b%Y', errors='coerce')
).dropna() > pd.Timestamp('2024-12-31')

# Check if all correctly converted dates (excluding 'NaT') are after the year 2024. This verification step helps in
# identifying whether there are any dates that are implausibly far in the future, which could indicate data entry errors
# or placeholders.
all_dates_after_2024 = dates_after_2024.all()

print("Are all 'Var13' dates after conversion, that aren't 'NaT', beyond the year 2024?:", all_dates_after_2024)
# This indicates we have 784 cases for 'Var13' where the dates are likely placeholders.


Original 'Var13' values that became empty after conversion:
5        31Dec9999
56       31Dec9999
67       31Dec9999
86       31Dec9999
95       31Dec9999
           ...    
49510    31Dec9999
49537    31Dec9999
49617    31Dec9999
49635    31Dec9999
49765    31Dec9999
Name: Var13, Length: 784, dtype: object
Are all 'Var13' dates after conversion, that aren't 'NaT', beyond the year 2024?: True


In [385]:
# The objective is to create a new column named 'working_months'. This column calculates the number of months
# an employee has been working in their current position from the time of employment until the credit application date.
# This can provide insights into work stability or tenure when evaluating credit applications.

# Creating the 'working_months' column
# We calculate the difference between 'application_date' and 'Var13' (the employment start date) in months.
# The calculation involves subtracting 'Var13' from 'application_date', then dividing by the numpy timedelta object for one month ('M')
# to convert the difference into months. The result may have floating point values, so we fill any missing values with 0
# and convert the series to integers for ease of interpretation.
sorted_data['working_months'] = ((sorted_data['application_date'] - sorted_data['Var13']) / np.timedelta64(1, 'M')).fillna(0).astype(int)

# Removing the 'application_date' and 'Var13' columns
# Since we've captured the essential information in 'working_months', we remove these columns to simplify the dataset.
sorted_data.drop(['application_date', 'Var13'], axis=1, inplace=True)

# Setting 'working_months' as the first column in the DataFrame
# We first compile a list of all columns excluding 'working_months' to avoid duplication.
columns_without_working_months = [col for col in sorted_data.columns if col != 'working_months']

# We then create a new list of columns with 'working_months' positioned at the start.
new_columns_order = ['working_months'] + columns_without_working_months

# Reorganize the DataFrame according to the new column order.
# This step ensures 'working_months' is the first column, making it more prominent when the data is viewed or analyzed.
sorted_data = sorted_data.reindex(columns=new_columns_order)

sorted_data.head()






C:\Users\jansl\AppData\Local\Temp\ipykernel_11088\1534076300.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data['working_months'] = ((sorted_data['application_date'] - sorted_data['Var13']) / np.timedelta64(1, 'M')).fillna(0).astype(int)
C:\Users\jansl\AppData\Local\Temp\ipykernel_11088\1534076300.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data.drop(['application_date', 'Var13'], axis=1, inplace=True)


,working_months,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,62,11034977,32537148,0.267045,5640,NaN,1,7800,99,1,...,1,5,NaN,2,0.0,Approved,1.0,NaN,0,0
1,160,11034978,32761663,0.533040,3000,NaN,1,11100,78,1,...,2,2,NaN,4,0.0,Approved,NaN,1.0,0,0
2,248,11034979,32701063,0.861555,6240,4300.0,2,2400,15,1,...,1,5,5.0,1,0.0,Approved,NaN,NaN,0,0
3,265,11034980,32386786,0.506451,4560,2500.0,3,11800,30,6,...,2,6,5.0,1,0.0,Approved,NaN,0.0,0,0
4,248,11034981,32692110,0.210913,5160,NaN,1,10200,72,3,...,1,5,NaN,0,NaN,Rejected,1.0,NaN,0,0


### Binary convertion for 'Application_status'

In [386]:
# Convert the 'Application_status' column to binary variables to facilitate analysis.
# Applications marked as 'Approved' are changed to 1, and those marked as 'Rejected' are changed to 0.
# This conversion simplifies the analysis process, allowing for straightforward interpretation of application outcomes.
sorted_data['Application_status'] = sorted_data['Application_status'].replace({'Approved': 1, 'Rejected': 0})

# Check the first few rows of the 'Application_status' column to ensure the conversion has been applied correctly.
# This step is crucial for verifying that the transformation was successful and the data is ready for further analysis.
sorted_data[['Application_status']].head()


,Application_status
0,1
1,1
2,1
3,1
4,0


### One-hot encoding for nominal categorical variables

In [387]:
# Apply one-hot encoding to the specified categorical nominal variables in the 'sorted_data' DataFrame.
# The variables identified for this encoding are: Var2, Var3, Var11, Var12, and Var14.
# One-hot encoding converts these categorical variables into a form that could be provided to ML algorithms
# to do a better job in prediction. It creates binary columns for each category/value in the original columns
# and assigns a 1 or 0 (presence or absence) in those newly created binary columns.

# The 'pd.get_dummies()' function is used here to perform one-hot encoding on the specified columns.
# It automatically generates new columns for each unique value in the specified categorical columns,
# effectively transforming them into a format that's suitable for modeling.
sorted_data_encoded = pd.get_dummies(sorted_data, columns=['Var2', 'Var3', 'Var11', 'Var12', 'Var14'])

# Display the first few rows of the DataFrame to check the results of the encoding.
# This step allows us to verify that the one-hot encoding process has successfully transformed the specified
# categorical variables into a set of binary variables, each representing a unique category within the original variables.
sorted_data_encoded.head()



,working_months,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,...,Var12_3.0,Var12_4.0,Var12_5.0,Var12_6.0,Var12_7.0,Var14_0,Var14_1,Var14_2,Var14_3,Var14_4
0,62,11034977,32537148,0.267045,5640,NaN,1,7800,99,1,...,0,0,0,0,0,0,0,1,0,0
1,160,11034978,32761663,0.533040,3000,NaN,1,11100,78,1,...,0,0,0,0,0,0,0,0,0,1
2,248,11034979,32701063,0.861555,6240,4300.0,2,2400,15,1,...,0,0,1,0,0,0,1,0,0,0
3,265,11034980,32386786,0.506451,4560,2500.0,3,11800,30,6,...,0,0,1,0,0,0,1,0,0,0
4,248,11034981,32692110,0.210913,5160,NaN,1,10200,72,3,...,0,0,0,0,0,1,0,0,0,0


### Null analysis per ID 

In [388]:
# First, we create a copy of our DataFrame and set the 'ID' column as the index. This step makes it easier to perform operations
# that are specific to each record based on its unique identifier.
data_l = data.copy()
data_l.set_index('ID', inplace=True)

# Next, we calculate the number of null values in each row. We also list the columns that contain these nulls.
# This is done by applying a lambda function across each row (axis=1), which sums up the nulls (`x.sum()`)
# and identifies which columns they are in (`list(data_l.columns[x])`). The results are stored in a Series
# with two indices: 'Null_Count' for the number of nulls, and 'Null_Columns' for the specific columns with null values.
null_analysis_per_id = data_l.isnull().apply(lambda x: pd.Series([x.sum(), list(data_l.columns[x])], index=['Null_Count', 'Null_Columns']), axis=1)

# Display the analysis results. This provides insight into how null values are distributed across the dataset,
# including which records are affected and how.
null_analysis_per_id

# A potential issue arises when dealing with one-hot encoding of nominal categorical variables that had no missing values.
# During one-hot encoding, these variables are transformed such that each category becomes a separate column,
# with rows marked as 0 to indicate the absence of that category. This automatic conversion to 0 for all categories
# in a one-hot encoded scheme can lead to misinterpretation when assessing null values.
# Moreover, it's noted that every record in the dataset has at least one null value, which complicates the analysis
# of data completeness and requires careful consideration when preparing the data for modeling or analysis.

# This highlights the importance of understanding how different data preprocessing steps, like one-hot encoding,
# can affect the interpretation of null values in the dataset. It's crucial to distinguish between genuinely missing values
# and the artificial zeros introduced by one-hot encoding, as they represent different aspects of the data.



,Null_Count,Null_Columns
ID,,
11034977,4,"[Var8, Var10, Var12, Var19]"
11034978,3,"[Var10, Var12, Var18]"
11034979,3,"[Var8, Var18, Var19]"
11034980,1,[Var18]
11034981,5,"[target, Var8, Var10, Var12, Var19]"
...,...,...
11084972,3,"[Var18, Var25, Var26]"
11084973,7,"[target, Var8, Var10, Var12, Var19, Var25, Var26]"
11084974,3,"[Var10, Var12, Var18]"


### Data cleaning for numeric data types 

In [389]:
# Convert binary variables to the 'Int8' data type, which supports null values.
# This step is important for maintaining data integrity, as 'Int8' allows for binary data to include
# missing values, offering a more accurate representation of the dataset where nulls are meaningful.
binary_cols = ['target','Application_status', 'Var18', 'Var19', 'Var27', 'Var28']
sorted_data_encoded[binary_cols] = sorted_data_encoded[binary_cols].astype('Int8')

# Import necessary libraries for KNN imputation.
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd

# Save the original data types of the DataFrame. This is crucial for restoring the DataFrame's data types
# after imputation since KNNImputer will convert the imputed data to float64 by default.
original_dtypes = sorted_data_encoded.dtypes

# Initialize the KNNImputer. The default setting uses 5 nearest neighbors to impute missing values,
# which is a balance between accuracy and computational efficiency.
knn_imputer = KNNImputer()

# Replace 0s with NaNs in the 'working_months' column if necessary. This step is preparatory for imputation,
# ensuring that 0s, which may signify missing or unrecorded data, are correctly treated as missing values.
sorted_data_encoded['working_months'] = sorted_data_encoded['working_months'].replace(0, np.nan)

# Select only numerical columns from the DataFrame for imputation. KNNImputer works with numerical data,
# so identifying these columns is a prerequisite for the imputation process.
numerical_cols = sorted_data_encoded.select_dtypes(include=['int64', 'float64', 'int32']).columns

# Perform the imputation using KNN. This process fills in missing values based on the mean of the
# nearest neighbors found in the multidimensional space, providing a reasonable estimate for missing data.
imputed_data = knn_imputer.fit_transform(sorted_data_encoded[numerical_cols])

# Convert the imputed array back into a DataFrame, preserving the original index. This step ensures that
# the imputed data retains the structure and index of the original DataFrame for consistency and ease of analysis.
imputed_df = pd.DataFrame(imputed_data, columns=numerical_cols, index=sorted_data_encoded.index)

# Restore the original data types. After imputation, it's important to convert the data back to its
# original types for accurate analysis and consistency with the rest of the dataset.
for col in numerical_cols:
    if original_dtypes[col].kind in 'iu':  # Check if the data type is integer
        imputed_df[col] = imputed_df[col].round().astype(original_dtypes[col])
    else:  # For floating-point types, retain as is
        imputed_df[col] = imputed_df[col].astype(original_dtypes[col])

# Update the original 'sorted_data_encoded' DataFrame with the imputed data. This final step integrates
# the imputed numerical data back into the main DataFrame, completing the imputation process.
sorted_data_encoded.update(imputed_df)

# Display the DataFrame information to check the results of the imputation and data type restoration.
# This overview allows for a quick verification that the dataset is now complete with imputed values
# and that all columns are of their appropriate data types.
sorted_data_encoded.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 57 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   working_months      50000 non-null  float64
 1   ID                  50000 non-null  int64  
 2   customer_id         50000 non-null  int64  
 3   _r_                 50000 non-null  float64
 4   Var9                50000 non-null  int64  
 5   Var10               50000 non-null  float64
 6   Var1                50000 non-null  int64  
 7   Var4                50000 non-null  int64  
 8   Var5                50000 non-null  int64  
 9   Var6                50000 non-null  int64  
 10  Var7                50000 non-null  float64
 11  Var8                50000 non-null  float64
 12  Var15               50000 non-null  int64  
 13  Var16               50000 non-null  int64  
 14  Var17               50000 non-null  float64
 15  Var20               50000 non-null  int64  
 16  Var2

### Data cleaning for binary data types (not for one hot encoding nominal categorical variables, after decomposition)

In [390]:
# Handling missing values in binary columns
# This section focuses on imputing missing values in binary variables using the mode.
# The mode is the most common value in a distribution, making it a suitable choice for imputing
# binary variables as it reflects the predominant category.

# List of binary columns identified for mode imputation.
binary_cols = ['target','Application_status', 'Var18', 'Var19', 'Var27', 'Var28']

# Loop through each binary column to perform mode imputation.
for col in binary_cols:
    # Calculate the mode for each binary column. The 'dropna=True' parameter ensures that missing values
    # are not considered when determining the mode. '[0]' is used to select the first mode in case there
    # are multiple modes with the same frequency.
    mode_value = sorted_data_encoded[col].mode(dropna=True)[0]
    
    # Impute missing values with the mode. The 'fillna()' function replaces all NaNs in the column with
    # the mode value calculated previously. 'inplace=True' modifies the original DataFrame directly,
    # saving the need to create a copy or reassign the DataFrame.
    sorted_data_encoded[col].fillna(mode_value, inplace=True)

# After imputing missing values in the binary columns, it's a good practice to verify the changes.
# Checking that there are no more missing values in these columns can be done through the 'info()' method
# or specifically checking for nulls using 'isnull().sum()' on the binary columns.



